In [3]:
import torch
import torch.nn as nn


In [ ]:
from argparse import Namespace

modelArgs = Namespace(
    
data_csv = 'timeSeriesData.csv',
model_state_file = "model.pth",
seed = 1300,
learning_rate = 0.001,
# dropout_p = 0.1,
batch_size = 128,
num_epochs = 40,
cuda = True,

);

In [13]:
class Rnn(nn.Module):

    def __init__(self,input_size , hidden_size , output_size):

        super(Rnn,self).__init__();
        self.hidden_size = hidden_size;
        self.input_size = input_size;
        self.output_size = output_size;
        self.i2h = nn.Linear(self.input_size + self.hidden_size , self.hidden_size);
        self.i2o = nn.Linear(self.input_size + self.hidden_size , self.output_size);
        self.sigmoid = nn.Sigmoid();
    
    def forward(self,input_tensor,hidden_tensor):
        print(f'input_tensor {input_tensor} with size == {input_tensor.size()} \n');
        print(f'hidden_tensor {hidden_tensor} with size == {hidden_tensor.size()} \n');

        combined = torch.cat((input_tensor , hidden_tensor) , 1);
        print(f'combined_tensor {combined} with size == {combined.size()} \n');
        hidden = self.i2h(combined)
        output = self.sigmoid(self.i2o(combined));
        return output,hidden;


    def init_hidden(self):
        return torch.zeros(1,self.hidden_size);
    



rnn = Rnn(20 , 128 , 1)
hidden_tensor = rnn.init_hidden();
input_tensor = torch.ones(1,20); 
output, next_hidden = rnn(input_tensor , hidden_tensor);
print(output,next_hidden)


input_tensor tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]]) with size == torch.Size([1, 20]) 

hidden_tensor tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]]) with size == torch.Size([1, 128]) 

combined_tensor tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,